In [19]:
import pandas as pd
import os
import json
import numpy as np
from datetime import datetime

In [20]:
path_folder = r'C:\Users\Win10\End-to-End-Testing-main\datacrawl'
datas = os.listdir(path_folder)
for d in datas:
    print(d)

dprnews.json
dprtwitter.json
dpryt.json


In [21]:
path_news = os.path.join(path_folder, datas[0])

with open(path_news, 'r') as f:
    news_dat = json.load(f)

In [22]:
news_df = pd.DataFrame(news_dat['articles'])
print(len(news_df))

27


In [23]:
news_df['label_owner'] = 'Personal'
news_df['label_timestamp'] = datetime.now().isoformat()

news_df.head(3)

,judul,ringkasan,sumber,tanggal,url,label_owner,label_timestamp
0,"Profil Hetifah Sjaifudian, Anggota DPR RI Dapi...",Sebanyak 580 caleg berhasil memastikan diri lo...,Tribun Kaltim,2024-08-02T02:53:20Z,https://kaltim.tribunnews.com/2024/08/02/profi...,Personal,2024-08-02T11:16:59.615050
1,Menhub Budi Pede Tol Laut Lanjut di Masa Prabo...,Budi meyakini bahwa program tol laut tidak han...,VIVA.co.id,2024-08-02T02:41:08Z,https://www.viva.co.id/bisnis/1738349-menhub-b...,Personal,2024-08-02T11:16:59.615050
2,Sah! Pasangan Salim Fakhry-Heri Al Hilal Maju ...,"KUTACANE, Waspada.co.id - Dewan Pimpinan Pusat...",Waspada Online,2024-08-02T02:40:57Z,https://waspada.co.id/sah-pasangan-salim-fakhr...,Personal,2024-08-02T11:16:59.615050


In [24]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')  # Ganti dengan URI MongoDB Anda
db = client['TESTING'] 
collection = db['search_result']

In [25]:
# Convert DataFrame to dictionary
data_dict = news_df.to_dict("records")


In [27]:
# Fetch existing data from 'news' document
existing_document = collection.find_one({'_id': 'news'})

# Check for duplicates based on 'url' and append new data
if existing_document and 'data' in existing_document:
    existing_data = existing_document['data']
    existing_urls = {item['url'] for item in existing_data}
    
    # Filter out new items that already exist based on the 'url' field
    new_data = [item for item in data_dict if item['url'] not in existing_urls]
    
    if new_data:
        existing_data.extend(new_data)
        collection.update_one(
            {'_id': 'news'},
            {'$set': {'data': existing_data}}
        )
else:
    collection.update_one(
        {'_id': 'news'},
        {'$set': {'data': data_dict}},
        upsert=True
    )